In [1]:
!pip install requests

In [9]:
import requests
import json
import time

# --- CONFIGURATION ---
API_KEY = "bOWk8GBfZ9aVLT6hOkIZEuzd"
TOTAL_GOAL = 10000
OUTPUT_FILE = "tech_gaming_dataset_10000.json"

# Expanded Category IDs for Tech & Gaming:
# abcat0501000: Laptops
# pcmcat209400050001: Smartphones
# abcat0515000: Computer Accessories & Gadgets
# abcat0811002: Cell Phone Accessories
# pcmcat1476977531756: Gaming Consoles (PS5, Xbox, Switch)
# abcat0507000: PC Components (GPUs, CPUs)
# pcmcat242800050021: Wearables (Smartwatches)
# abcat0204000: Headphones/Audio
# pcmcat209400050001: Mobile Phones

CATEGORIES = (
    "(categoryPath.id in(abcat0501000,pcmcat209400050001,abcat0515000,"
    "abcat0811002,pcmcat1476977531756,abcat0507000,pcmcat242800050021,abcat0204000))"
)

def fetch_10000_tech_products():
    all_products = []
    page = 1

    print(f"🚀 Starting 10k Collection...")

    while len(all_products) < TOTAL_GOAL:
        url = (
            f"https://api.bestbuy.com/v1/products{CATEGORIES}?"
            f"apiKey={API_KEY}&format=json&pageSize=100&page={page}"
            f"&show=name,salePrice,url,image,features.feature"
        )

        try:
            response = requests.get(url)

            if response.status_code in [403, 429]:
                print(f"⚠️ Rate limit. Sleeping 20s...")
                time.sleep(20)
                continue

            response.raise_for_status()
            data = response.json()

            products = data.get("products", [])
            if not products:
                print("🏁 No more unique products found in these categories.")
                break

            for item in products:
                if len(all_products) >= TOTAL_GOAL:
                    break

                all_products.append({
                    "id": len(all_products) + 1,
                    "title": item.get("name"),
                    "price": item.get("salePrice"),
                    "link": item.get("url"),
                    "image": item.get("image"),
                    "characteristics": [f.get("feature") for f in item.get("features", [])]
                })

            print(f"✅ Progress: {len(all_products)}/{TOTAL_GOAL} (Page {page})")

            # Optimized speed: 0.5s is usually safe for Best Buy's 5-calls-per-second limit
            time.sleep(0.5)
            page += 1

            # Auto-save every 1000 items so you don't lose data
            if len(all_products) % 1000 == 0:
                with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
                    json.dump(all_products, f, indent=4)

        except Exception as e:
            print(f"❌ Error: {e}. Retrying...")
            time.sleep(5)

    # Final Save and Close
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(all_products, f, indent=4)

    print(f"\n🎉 SUCCESS! Target reached. {len(all_products)} items saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    fetch_10000_tech_products()

🚀 Starting 10k Collection...
✅ Progress: 100/10000 (Page 1)
✅ Progress: 200/10000 (Page 2)
✅ Progress: 300/10000 (Page 3)
✅ Progress: 400/10000 (Page 4)
✅ Progress: 500/10000 (Page 5)
✅ Progress: 600/10000 (Page 6)
⚠️ Rate limit. Sleeping 20s...
✅ Progress: 700/10000 (Page 7)
✅ Progress: 800/10000 (Page 8)
✅ Progress: 900/10000 (Page 9)
✅ Progress: 1000/10000 (Page 10)
✅ Progress: 1100/10000 (Page 11)
✅ Progress: 1200/10000 (Page 12)
✅ Progress: 1300/10000 (Page 13)
✅ Progress: 1400/10000 (Page 14)
⚠️ Rate limit. Sleeping 20s...
✅ Progress: 1500/10000 (Page 15)
⚠️ Rate limit. Sleeping 20s...
✅ Progress: 1600/10000 (Page 16)
⚠️ Rate limit. Sleeping 20s...
✅ Progress: 1700/10000 (Page 17)
✅ Progress: 1800/10000 (Page 18)
✅ Progress: 1900/10000 (Page 19)
✅ Progress: 2000/10000 (Page 20)
✅ Progress: 2100/10000 (Page 21)
✅ Progress: 2200/10000 (Page 22)
✅ Progress: 2300/10000 (Page 23)
✅ Progress: 2400/10000 (Page 24)
✅ Progress: 2500/10000 (Page 25)
⚠️ Rate limit. Sleeping 20s...
✅ Progres

In [22]:
import json
import html
import re

def clean_and_format_dataset(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        processed_data = []

        for item in data:
            # 1. Decode HTML entities (e.g., &#179; -> ³)
            title = html.unescape(item.get("title", ""))

            # Clean and unescape characteristics
            raw_chars = [html.unescape(str(c)) for c in item.get("characteristics", [])]

            # 2. Clean noise (removing superscript footnote numbers)
            # This makes the text "cleaner" for the embedding model to read
            clean_chars = [re.sub(r'[¹²³⁴⁵⁶⁷⁸⁹⁰]', '', char).strip() for char in raw_chars]

            # 3. Concatenate title + all characteristics into one single paragraph
            # We start with the title to give the vector context
            full_description = f"{title}. " + " ".join(clean_chars)

            # Remove excessive whitespace/newlines within the paragraph
            full_description = re.sub(r'\s+', ' ', full_description).strip()

            # 4. Construct the optimized record
            optimized_item = {
                "id": item.get("id"),
                "title": title,
                "price": item.get("price"),
                "link": item.get("link"),  # Kept exactly as original
                "image": item.get("image"),
                "characteristics": full_description
            }
            processed_data.append(optimized_item)

        # 5. Save the final JSON with the correct parameter
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(processed_data, f, indent=4, ensure_ascii=False)

        print(f"Success! Processed {len(processed_data)} items into {output_file}")

    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Execute
clean_and_format_dataset('tech_gaming_dataset_10000.json', 'qdrant_optimized_dataset.json')

Success! Processed 7327 items into qdrant_optimized_dataset.json


In [24]:
import os
import json
import requests
import time
from pathlib import Path

# Configuration
INPUT_FILE = 'qdrant_optimized_dataset.json'
OUTPUT_FILE = 'qdrant_local_dataset.json'
IMAGE_FOLDER = 'downloaded_images'

# Create the folder if it doesn't exist
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

# Realistic headers to mimic a real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Accept": "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.bestbuy.com/"
}

def download_images_and_update_json():
    with open(INPUT_FILE, 'r') as f:
        products = json.load(f)

    updated_products = []
    print(f"Starting stealth download for {len(products)} items...")

    # Use a session for better performance and persistence
    session = requests.Session()
    session.headers.update(HEADERS)

    for item in products:
        product_id = item.get('id')
        image_url = item.get('image')

        local_filename = f"product_{product_id}.jpg"
        local_path = os.path.join(IMAGE_FOLDER, local_filename)

        # Skip if already downloaded (saves time if you restart)
        if os.path.exists(local_path):
            item['image_local'] = str(Path(local_path).resolve())
            updated_products.append(item)
            continue

        try:
            response = session.get(image_url, stream=True, timeout=15)

            if response.status_code == 200:
                with open(local_path, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                item['image_local'] = str(Path(local_path).resolve())
                print(f"✅ Success: Item {product_id}")

                # IMPORTANT: Wait 1 second between downloads to avoid being banned again
                time.sleep(1)

            else:
                print(f"❌ Failed: Item {product_id} (HTTP {response.status_code})")
                item['image_local'] = None

        except Exception as e:
            print(f"⚠️ Error on item {product_id}: {e}")
            item['image_local'] = None

        updated_products.append(item)

    with open(OUTPUT_FILE, 'w') as f:
        json.dump(updated_products, f, indent=4)

    print(f"\nDone! Dataset saved as {OUTPUT_FILE}")

if __name__ == "__main__":
    download_images_and_update_json()

Streaming output truncated to the last 5000 lines.
✅ Success: Item 2330
✅ Success: Item 2331
✅ Success: Item 2332
✅ Success: Item 2333
✅ Success: Item 2334
✅ Success: Item 2335
✅ Success: Item 2336
✅ Success: Item 2337
✅ Success: Item 2338
✅ Success: Item 2339
✅ Success: Item 2340
✅ Success: Item 2341
✅ Success: Item 2342
✅ Success: Item 2343
✅ Success: Item 2344
✅ Success: Item 2345
✅ Success: Item 2346
✅ Success: Item 2347
✅ Success: Item 2348
✅ Success: Item 2349
✅ Success: Item 2350
✅ Success: Item 2351
✅ Success: Item 2352
✅ Success: Item 2353
✅ Success: Item 2354
✅ Success: Item 2355
✅ Success: Item 2356
✅ Success: Item 2357
✅ Success: Item 2358
✅ Success: Item 2359
✅ Success: Item 2360
✅ Success: Item 2361
✅ Success: Item 2362
✅ Success: Item 2363
✅ Success: Item 2364
✅ Success: Item 2365
✅ Success: Item 2366
✅ Success: Item 2367
✅ Success: Item 2368
✅ Success: Item 2369
✅ Success: Item 2370
✅ Success: Item 2371
✅ Success: Item 2372
✅ Success: Item 2373
✅ Success: Item 2374
✅ Su

In [25]:
!zip -r /content/folder_name.zip /content/downloaded_images

Streaming output truncated to the last 5000 lines.
  adding: content/downloaded_images/product_679.jpg (deflated 3%)
  adding: content/downloaded_images/product_1071.jpg (deflated 4%)
  adding: content/downloaded_images/product_1535.jpg (deflated 1%)
  adding: content/downloaded_images/product_3819.jpg (deflated 28%)
  adding: content/downloaded_images/product_5462.jpg (deflated 7%)
  adding: content/downloaded_images/product_6106.jpg (deflated 1%)
  adding: content/downloaded_images/product_4927.jpg (deflated 15%)
  adding: content/downloaded_images/product_5859.jpg (deflated 11%)
  adding: content/downloaded_images/product_3551.jpg (deflated 6%)
  adding: content/downloaded_images/product_1389.jpg (deflated 13%)
  adding: content/downloaded_images/product_2777.jpg (deflated 4%)
  adding: content/downloaded_images/product_3003.jpg (deflated 3%)
  adding: content/downloaded_images/product_2138.jpg (deflated 20%)
  adding: content/downloaded_images/product_5028.jpg (deflated 13%)
  addi